In [28]:
import pandas as pd
import datetime
import numpy as np
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pickle

In [29]:
train1 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
train = train[train.notnull().apply(all, axis=1)]
dev = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_dev.csv')
dev = dev[dev.notnull().apply(all, axis=1)]
test = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_test.csv')
test = test[test.notnull().apply(all, axis=1)]

dev_prepared = pd.read_csv('../02_Extract_Subphrases/prepared_data/subphrase_vectors_dev.csv', sep=';')
dev_prepared = dev_prepared.drop(columns='Unnamed: 0')
dev = dev.set_index('pairID')
y_hat = dev.loc[dev_prepared.iloc[:,0]].gold_label
dev_prepared = dev_prepared.iloc[:,1:].to_numpy()

In [30]:
def evaluate_performance(preds, actual):
    labels = ['contradiction', 'entailment', 'neutral']
    print('Confusion Matrix')
    print(confusion_matrix(actual, preds, labels=labels))
    print(f'F1-Score: {f1_score(actual, preds, labels=labels, average="macro")}')
    print(f'Accuracy: {accuracy_score(actual, preds)}')

In [31]:
def predict_y_from_z(z):
    if len(z.shape) > 1:
        z = pd.DataFrame(z)
        res = z.apply(predict_y_from_z, axis=1)
        return res.to_numpy()
    else:
        if all([z[i] == 'nan' or pd.isnull(z[i]) or z[i] == 'entailment' for i in range(25)]):
            return 'entailment'
        elif any(z == 'contradiction'):
            return 'contradiction'
        elif any(z == 'neutral') and all(z != 'contradiction'):
            return 'neutral'
        else:
            raise ValueError(f"z can only have values 'entailment', 'contradiction', or 'neutral' but is {z}")

In [32]:
directory = 'MLP_Classifiers_480k_training_60_iter_NN_size_200_200_200_50_30_30'
clf = list()
for i in range(25):
    with open(directory + "/MLP_Classifier" + str(i) + ".pkl", "rb") as f:
        clf += [pickle.load(f), ]

In [33]:
# Prepare colum indices
indices = [[0,1500], [0,1800], [0,2100], [0,2400], [0,2700],
           [300,1500], [300,1800], [300,2100], [300,2400], [300,2700],
           [600,1500], [600,1800], [600,2100], [600,2400], [600,2700],
           [900,1500], [900,1800], [900,2100], [900,2400], [900,2700],
           [1200,1500], [1200,1800], [1200,2100], [1200,2400], [1200,2700]]

# Initialise colulmn indices and "nan" values if information (e.g. location of sentence) is not detected
print(datetime.datetime.now())
print("Initialise column indices and 'nan' values")
not_nan = [None, ] * 25
cols = [None, ] * 25
for i in range(25):
    cols[i] = list(range(indices[i][0], indices[i][0]+300)) + list(range(indices[i][1],indices[i][1]+300))
    not_nan[i] = pd.Series([not x for x in pd.DataFrame(np.isnan(dev_prepared[:,cols[i]])).apply(any, axis=1)])
not_nan = np.array(not_nan).T

2023-03-09 09:18:59.921012
Initialise column indices and 'nan' values


In [34]:
z = np.empty((y_hat.shape[0], 25), dtype=np.dtype('U20'))
z[:,:] = np.nan

for i in range(25):
    z[not_nan[:,i], i] = clf[i].predict(dev_prepared[not_nan[:,i],:][:, cols[i]])

y_hat_pred = predict_y_from_z(z)

In [35]:
z

array([['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan'],
       ['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan'],
       ['entailment', 'entailment', 'nan', ..., 'nan', 'nan', 'nan'],
       ...,
       ['entailment', 'entailment', 'nan', ..., 'nan', 'nan', 'nan'],
       ['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan'],
       ['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan']], dtype='<U20')

In [36]:
y_hat_pred

array(['neutral', 'neutral', 'entailment', ..., 'entailment', 'neutral',
       'contradiction'], dtype=object)

In [37]:
y_hat

pairID
4705552913.jpg#2r1n          neutral
4705552913.jpg#2r1e       entailment
4705552913.jpg#2r1c    contradiction
3948003394.jpg#1r1e       entailment
3948003394.jpg#1r1c    contradiction
                           ...      
3344632789.jpg#0r2c    contradiction
3344632789.jpg#0r2n          neutral
3344632789.jpg#0r1e       entailment
3344632789.jpg#0r1c    contradiction
3344632789.jpg#0r2e       entailment
Name: gold_label, Length: 3035, dtype: object

In [21]:
(y_hat_pred == 'neutral').mean()

0.32520593080724874

In [22]:
(y_hat == 'neutral').mean()

0.31960461285008235

In [38]:
evaluate_performance(y_hat_pred, y_hat)

Confusion Matrix
[[392 373 280]
 [167 606 247]
 [165 380 425]]
F1-Score: 0.46496455199762066
Accuracy: 0.4688632619439868


In [144]:
clf[0].predict(dev_prepared[not_nan[:,0],:][:, cols[0]])

array(['neutral', 'neutral', 'neutral', ..., 'neutral', 'neutral',
       'neutral'], dtype='<U13')